In [290]:
import pandas as pd

def get_top_usluga_url(url:str) -> str:
    return url[:url.rfind('/')]

def get_relative_url(url:str) -> str:
    return url.replace('https://ddcar.ru', '')

def get_end_usluga_url(url:str) -> str:
    return url[url.rfind('/'):]


usluga = 'Замена топливного фильтра'
usluga_url = '/remont-dvigatelya/zamena-toplivnogo-filtra'    

data_read = pd.read_csv('marka_model.csv', encoding = 'utf-8-sig', sep=';')
# добавляем столбец с услугой к данным
data_read['Услуга'] = usluga
db_all_page = pd.read_csv('export_20200213-103536.csv', encoding = 'cp1251', sep=';')
print(db_all_page['Основной текст'])
db_all_page = db_all_page.iloc[:, :6]

#находим родительский раздел, в котором лежит услуга
top_usluga_url = get_top_usluga_url(usluga_url)

#ищем раздел в выгрузке с пересечением марки и модели

for idx, data in data_read.iterrows():
    data_filter_marka = db_all_page[db_all_page['Марка'] == data.Марка]
    if pd.notna(data.Модель):
        data_filter_model = data_filter_marka[data_filter_marka['Модель'] == data.Модель]
    else:
        data_filter_model = data_filter_marka[pd.isna(data_filter_marka['Модель'])]
    data_filter_model_url = data_filter_model[data_filter_model['_URL_'].str.endswith(f'{top_usluga_url}')]
    if len(data_filter_model_url):
        url_change = data_filter_model_url._URL_ + get_end_usluga_url(usluga_url)
        print(url_change)
        data_read.loc[idx, ['URL']] = url_change.item()
    else:
        print(f'Не найдена услуга {top_usluga_url} для \n{data}')
        data_read.loc[idx, ['URL']] = get_relative_url(data.URL + usluga_url)


print(data_read['URL'])
data_read.to_csv('file_import_with_chek_for_top_usluga.csv', sep= ";", encoding = "utf-8-sig")

#выбираем из общей выгрузки те, что совпадают по маркам+моделям
#db_all_page = db_all_page.loc[db_all_page['Марка'] == data['Марка'][1]]
#db_all_page = db_all_page.loc[db_all_page['Модель'] == data['Модель'][1]]

#db_all_page = db_all_page.loc[db_all_page['_URL_'] == data['Модель'][1]]
#data = data.loc[data['Марка'] == data['Марка'][1]]
#data = data.loc[db_all_page['Модель'] == data['Модель'][1]]



0                    <p>Полный текст Ауди в новокосино</p>
1        <p>Заправка кондиционера BMW – одна из самых п...
2                                                      NaN
3        <h2>Особенности техобслуживания BMW X5</h2><p>...
4        <h2>Особенности ТО БМВ\r\n</h2>\r\n<p>Регламен...
                               ...                        
14995    <p>Топливный фильтр не пропускает в систему вп...
14996    <p>Колодки &ndash; один из важнейших элементов...
14997    <p>Автоматическая коробка передач Ford Mustang...
14998    <p>Коробка передач является самым прочным узло...
14999    <p>С процедурой замены или ремонта глушителя F...
Name: Основной текст, Length: 15000, dtype: object
303    /acura/slesarnyy-remont/remont-dvigatelya/zame...
Name: _URL_, dtype: object
1780    /acura/csx/remont-dvigatelya/zamena-toplivnogo...
Name: _URL_, dtype: object
1781    /acura/mdx/remont-dvigatelya/zamena-toplivnogo...
Name: _URL_, dtype: object
1782    /acura/rdx/remont-dvigatelya/zamena-to

C:\Users\lalko\Anaconda3\envs\seo\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: `item` has been deprecated and will be removed in a future version



996    /bmw/m5/slesarnyy-remont/remont-dvigatelya/zam...
Name: _URL_, dtype: object
997    /bmw/m6/slesarnyy-remont/remont-dvigatelya/zam...
Name: _URL_, dtype: object
998    /bmw/x5-m/slesarnyy-remont/remont-dvigatelya/z...
Name: _URL_, dtype: object
999    /bmw/x6-m/slesarnyy-remont/remont-dvigatelya/z...
Name: _URL_, dtype: object
1000    /bmw/z4-m/slesarnyy-remont/remont-dvigatelya/z...
Name: _URL_, dtype: object
1001    /bmw/1-seriya/slesarnyy-remont/remont-dvigatel...
Name: _URL_, dtype: object
1002    /bmw/2-seriya/slesarnyy-remont/remont-dvigatel...
Name: _URL_, dtype: object
1003    /bmw/2-seriya-active-tourer/slesarnyy-remont/r...
Name: _URL_, dtype: object
1004    /bmw/3-seriya/slesarnyy-remont/remont-dvigatel...
Name: _URL_, dtype: object
1005    /bmw/4-seriya/slesarnyy-remont/remont-dvigatel...
Name: _URL_, dtype: object
1006    /bmw/5-seriya/slesarnyy-remont/remont-dvigatel...
Name: _URL_, dtype: object
1007    /bmw/6-seriya/slesarnyy-remont/remont-dvigatel...
Name: _URL

ValueError: can only convert an array of size 1 to a Python scalar

In [188]:
d = pd.DataFrame()

print(d)

if len(d):
    print(2132131)
    
print(len(d))

Empty DataFrame
Columns: []
Index: []
0


In [175]:
db_all_page = pd.read_csv('export_20200213-103536.csv', encoding = 'cp1251', sep=';')
db_all_page1 = db_all_page[db_all_page['Марка'] == 'Acura']
print(db_all_page1['Модель'])
print(pd.notna(db_all_page1['Модель'][12178]))
if pd.notna(db_all_page1['Модель'][12178]):
    print(1)

21       NaN
22       NaN
23       NaN
24       NaN
25       NaN
        ... 
12178    ZDX
12179    ZDX
12180    ZDX
12181    ZDX
12182    ZDX
Name: Модель, Length: 248, dtype: object
True
1
